In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q tensorflow tensorflow-hub

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

module_path ="/kaggle/input/bigbigan/tensorflow1/resnet50/1"

# Load the BigBiGAN model from TensorFlow Hub
bigbigan = hub.load(module_path) 

In [ ]:
# Printing signatures to understand model strcuture 
print(bigbigan.signatures)

In [ ]:
import matplotlib.pyplot as plt

# Define dataset path
dataset_path = "/kaggle/input/cleandata/Road_Crack_Dataset_Cleaned_labled"

# Function to load images without labels
def load_images_from_directory(directory, batch_size=32):
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        directory,
        image_size=(128, 128),  # Resize to match BigBiGAN input
        batch_size=None,
        label_mode=None  # Ignore labels, only load images
    )
    return dataset

# Load datasets
train_dataset = load_images_from_directory(dataset_path + "/train/class_4")
val_dataset = load_images_from_directory(dataset_path + "/valid/class_4")
test_dataset = load_images_from_directory(dataset_path + "/test/class_4")

In [ ]:
import pathlib

# check images have been loaded properly 
subfolders = list(pathlib.Path(dataset_path + "/train").glob("*"))
print("Subfolders detected:", [folder.name for folder in subfolders])

# List some image files
image_files = list(pathlib.Path(dataset_path + "/train/class_0").glob("*.png"))[:5]
print("Sample images from class_0:", [img.name for img in image_files])

In [ ]:
import tensorflow as tf

def preprocess(image):
    # Ensure the image has 3 channels (RGB) if it's grayscale
    if len(image.shape) == 3 and image.shape[-1] == 1:  # Grayscale image (1 channel)
        image = tf.image.grayscale_to_rgb(image)  # Convert to RGB

    # Resize the image to 128x128
    image = tf.image.resize(image, [128, 128])
    
    # Normalize pixel values
    image = image / 255.0

    # Add the batch dimension, making it [1, 128, 128, 3]
    image = tf.expand_dims(image, axis=0)
    
    return image

# Apply preprocessing to the datasets
train_dataset = train_dataset.map(lambda x: preprocess(x)) \
    .shuffle(1000) \
    .batch(32, drop_remainder=True) \
    .prefetch(tf.data.AUTOTUNE)

val_dataset = val_dataset.map(lambda x: preprocess(x)) \
    .batch(32) \
    .prefetch(tf.data.AUTOTUNE)

test_dataset = test_dataset.map(lambda x: preprocess(x)) \
    .batch(32) \
    .prefetch(tf.data.AUTOTUNE)


In [ ]:
# Extract generator & encoder
generator = bigbigan.signatures['generate']
encoder = bigbigan.signatures['encode']

# Define optimizer & loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
loss_fn = tf.keras.losses.MeanSquaredError()

# Fine-tune the generator
# Modify the training loop to ensure the inputs are correctly formatted

def train_generator(epochs=10):
    for epoch in range(epochs):
        epoch_loss = 0
        for image_batch in train_dataset:
            with tf.GradientTape() as tape:
                # Squeeze the extra dimensions if they exist
                image_batch = tf.squeeze(image_batch, axis=1)  # Remove any singleton dimensions
                
                # Ensure the shape is [batch_size, 128, 128, 3]
                image_batch = tf.ensure_shape(image_batch, [None, 128, 128, 3])

                # Encode images to latent space
                z = encoder(image_batch)['default']
                reconstructed = generator(z)['default']  # Generate images
                loss = loss_fn(image_batch, reconstructed)  # Compute reconstruction loss

            gradients = tape.gradient(loss, generator.trainable_variables)
            optimizer.apply_gradients(zip(gradients, generator.trainable_variables))

            epoch_loss += loss.numpy()

        print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss / len(train_dataset)}')

train_generator(epochs=10)



In [ ]:
type(generator)

In [ ]:
# Unwrap the tf.function by getting the concrete function
concrete_function = generator.__call__.get_concrete_function()
tf.saved_model.save(concrete_function, '/kaggle/working/fine_tuned_generator')

In [ ]:
# Save the entire model
generator.save('/kaggle/working/fine_tuned_generator')

In [ ]:
generator_model.save('/kaggle/working/fine_tuned_generator')

In [ ]:
tf.saved_model.save(generator, '/kaggle/working/fine_tuned_generator')

In [ ]:
generator.load_weights('/kaggle/working/fine_tuned_generator_weights.h5')

In [ ]:
# Create a checkpoint object
checkpoint = tf.train.Checkpoint(generator=generator)

# Save the weights
checkpoint.save('/kaggle/working/fine_tuned_generator_ckpt')

In [ ]:
# Extract generator & encoder
generator = bigbigan.signatures['generate']
encoder = bigbigan.signatures['encode']

# Define optimizer & loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
loss_fn = tf.keras.losses.MeanSquaredError()

In [ ]:
# Define the checkpoint directory and base name
checkpoint_dir = '/kaggle/working'
checkpoint_prefix = f"{checkpoint_dir}/fine_tuned_generator_ckpt"

# Create a checkpoint object
checkpoint = tf.train.Checkpoint(generator=generator, optimizer=optimizer)

# Restore from the specific checkpoint if it exists
checkpoint_path = f"{checkpoint_prefix}-1"  # Use the base name without the file extensions
checkpoint.restore(checkpoint_path).expect_partial()
print(f"Restored from checkpoint: {checkpoint_path}")

# Continue fine-tuning
def train_generator(epochs=10):
    for epoch in range(epochs):
        epoch_loss = 0
        for image_batch in train_dataset:
            with tf.GradientTape() as tape:
                # Squeeze the extra dimensions if they exist
                image_batch = tf.squeeze(image_batch, axis=1)  # Remove any singleton dimensions
                
                # Ensure the shape is [batch_size, 128, 128, 3]
                image_batch = tf.ensure_shape(image_batch, [None, 128, 128, 3])

                # Encode images to latent space
                z = encoder(image_batch)['default']
                reconstructed = generator(z)['default']  # Generate images
                loss = loss_fn(image_batch, reconstructed)  # Compute reconstruction loss

            gradients = tape.gradient(loss, generator.trainable_variables)
            optimizer.apply_gradients(zip(gradients, generator.trainable_variables))

            epoch_loss += loss.numpy()

        print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss / len(train_dataset)}')

        # Save checkpoint after each epoch
        checkpoint.save(file_prefix=checkpoint_prefix)
        print(f"Checkpoint saved at {checkpoint_prefix}")

train_generator(epochs=10)

